In [ ]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr

# Analyzing Total Energy Consumption by State per Sector

In [ ]:
#Pull up Residential Energy Consumption throughout 50 states (including USA totals) dataframe from CSV file
df_res = pd.read_csv (r'Residential_Energy_Consumption_Updated.csv')
print (df_res)

In [ ]:
#Pull up Commercial and Industrial Energy Consumption throughout 50 states (including USA totals) dataframe
# from CSV file
df_com = pd.read_csv (r'Commercial_Energy_Consumption.csv')
print (df_com)

In [ ]:
#Pull up Commercial and Industrial Energy Consumption throughout 50 states (including USA totals) dataframe
# from CSV file
df_ind = pd.read_csv (r'Industrial_Energy_Consumption.csv')
print (df_ind)

In [ ]:
#Merge 3 DataFrames together
df_merged = pd.merge(df_res, df_com, on="State", suffixes=(None, "_Com"))
df_merged

In [ ]:
df_merged_final = pd.merge(df_merged, df_ind, on="State", suffixes=("_Res", "_Ind"))
df_merged_final = df_merged_final.drop([50])
df_merged_final

In [ ]:
#Create new DataFRame with just subtotals per sector
subtotal_df = df_merged_final[["State", "SubTotal_Res", "SubTotal_Com", "SubTotal_Ind"]]
subtotal_df["Total"]=(subtotal_df["SubTotal_Res"]+subtotal_df["SubTotal_Com"]+subtotal_df["SubTotal_Ind"])
subtotal_df = subtotal_df.sort_values("Total")
subtotal_df

In [ ]:
#Create a stacked bar chart to visualize energy consumption by sector per state

plt.figure(figsize=(20,15))

width = 0.35       # the width of the bars: can also be len(x) sequence
p1 = plt.bar(subtotal_df["State"], subtotal_df["SubTotal_Res"], width)
p2 = plt.bar(subtotal_df["State"], subtotal_df["SubTotal_Com"], width)
p3 = plt.bar(subtotal_df["State"], subtotal_df["SubTotal_Ind"], width)

plt.ylabel('Consumption')
plt.title('Energy Consumption by Sector per State')
plt.xticks(rotation='vertical')
plt.ylim(0,90000000)
# plt.yticks(np.arange(0, 81, 10))
plt.legend((p1[0], p2[0], p3[0]), ('Residential Sector', 'Commercial Sector', 'Industrial Sector' ))
plt.show()

In [ ]:
#Save the figure
plt.savefig('Figures/Energy_Consumption_Sector_State.png')

# Total Energy Consumption in the US from 2000 - 2015

In [ ]:
#Pull in total energy data files.  Need USA totals, so use the 'uncleaned files'
residential_df = pd.read_csv("data_output/Residential_Energy_Consumption.csv")
commercial_df = pd.read_csv("data_output/Commercial_Energy_Consumption.csv")
industrial_df = pd.read_csv("data_output/Industrial_Energy_Consumption.csv")

In [ ]:
#Set State column as Index
residential_df.set_index("State", inplace = True)
commercial_df.set_index("State", inplace = True)
industrial_df.set_index("State", inplace = True)

In [ ]:
#Get residential energy consumption from residential
res_US = residential_df.loc[residential_df.index == "USA"]
res_US_T = res_US.T
res_US_T.sort_index(inplace = True,ascending=True)
res_US_T

In [ ]:
#Get commercial energy consumption from USA
com_US = commercial_df.loc[commercial_df.index == "USA"]
com_US_T = com_US.T
com_US_T.sort_index(inplace = True,ascending=True)
com_US_T

In [ ]:
#Combine the two dataframes
combined_df = pd.merge(res_US_T, com_US_T, left_index = True, right_index = True)
combined_df

In [ ]:
#Get industrial energy from USA
ind_US = industrial_df.loc[industrial_df.index == "USA"]
ind_US_T = ind_US.T
ind_US_T.sort_index(inplace = True,ascending=True)
ind_US_T

In [ ]:
#Merge data frames
combined_df = pd.merge(combined_df, ind_US_T, left_index = True, right_index = True)
final_combined_df= combined_df.rename(columns = {'USA_x': 'Total Residential', "USA_y":"Total Commercial", "USA": "Total Industrial"})
final_combined_df

In [ ]:
#Sum up the individual sectors to get total consumption
final_combined_df["Total Consumption"] = final_combined_df["Total Residential"] + final_combined_df["Total Commercial"] + final_combined_df["Total Industrial"]

final_combined_df

In [ ]:
final_combined_df.describe()

In [ ]:
final_combined_df.plot(kind = "line", title = "Total Energy Consumption Across Three Sectors")
plt.xlabel("Year")
plt.ylabel("Consumption (Billion BTUs)")
plt.savefig("images/Fig1.png")
plt.show()

In [ ]:
total_consumption = final_combined_df["Total Consumption"]

total_consumption.plot(kind="line", title ="Total Energy Consumption Across Three Sectors", color = 'r')
plt.xlabel("Year")
plt.ylabel("Consumption (Billion BTUs)")
plt.savefig("images/Fig2.png")
plt.show()

In [ ]:
sectors_combined_df = final_combined_df.drop(columns = ['Total Consumption'])
sectors_combined_df.plot(kind="line", title ="Total Energy Consumption Across Three Sectors")
plt.xlabel("Year")
plt.ylabel("Consumption (Billion BTUs)")
plt.savefig("images/Fig4.png")
plt.show()

In [ ]:
sectors_combined_df["Total Residential"] /= 1000000
sectors_combined_df["Total Commercial"] /= 1000000
sectors_combined_df["Total Industrial"] /= 1000000
sectors_combined_df

In [ ]:
sectors_combined_df.plot(kind = "area", title = "Total Energy Consumption Across Three Sectors")
plt.xlabel("Year")
plt.ylabel("Consumption (Billion BTUs x10^6)")
plt.savefig("images/tot_energy_area.png")
plt.show()

In [ ]:
#Pie chart
residential_sum = final_combined_df["Total Residential"].sum()
commercial_sum =final_combined_df["Total Commercial"].sum()
industrial_sum = final_combined_df["Total Industrial"].sum()

# Labels for the sections of our pie chart
labels = ["Residential", "Commercial", "Industrial"]

# The values of each section of the pie chart
sizes = [residential_sum, commercial_sum, industrial_sum]

# The colors of each section of the pie chart
colors = ["blue", "orange", "green"]

plt.pie(sizes, labels=labels, colors=colors,
        autopct="%1.1f%%", shadow=True, startangle=140)
plt.title("Sectors Contributing to Total Energy Consumption in the USA")
plt.axis("equal")
plt.savefig("images/Fig5.png")
plt.show()

# CO2 Emissions

In [ ]:
#Get data from csv files
co2_residential_df = pd.read_csv("data_output/residential_c02.csv")
co2_commercial_df = pd.read_csv("data_output/commercial_c02.csv")
co2_industrial_df = pd.read_csv("data_output/industrial_c02.csv")

In [ ]:
co2_commercial_df.dtypes

In [ ]:
#Set 'States' as index
co2_residential_df.set_index("States", inplace = True)
co2_commercial_df.set_index("States", inplace = True)
co2_industrial_df.set_index("States", inplace = True)

#Sum values over each year column
sum_co2_residential = co2_residential_df.sum(axis=0)
sum_co2_commercial = co2_commercial_df.sum(axis=0)
sum_co2_industrial = co2_industrial_df.sum(axis=0)

In [ ]:
years = list(co2_residential_df.columns)
combined_em_df = pd.DataFrame({"Total Residential": sum_co2_residential,
                           "Total Commercial": sum_co2_commercial,
                           "Total Industrial": sum_co2_industrial})
combined_em_df["Total Emissions"] = combined_em_df["Total Residential"] + combined_em_df["Total Commercial"] + combined_em_df["Total Industrial"]
combined_em_df.dropna(inplace = True)
combined_em_df.dtypes

In [ ]:
combined_em_df.plot(kind = "line", title = "Total CO2 Emissions Across Three Sectors")
plt.xlabel("Year")
plt.ylabel("CO2 Emissions (million metric tons C02)")
plt.savefig("images/Fig6.png")
plt.show()

In [ ]:
total_emissions = combined_em_df["Total Emissions"]

total_emissions.plot(kind="line", title ="Total CO2 Emissions", color = 'r')
plt.xlabel("Year")
plt.ylabel("CO2 Emissions (million metric tons C02)")
plt.savefig("images/Fig7.png")
plt.show()

In [ ]:
em_sectors_combined_df = combined_em_df.drop(columns = ['Total Emissions'])
em_sectors_combined_df.plot(kind="line", title ="Total CO2 Emissions Across Three Sectors")
plt.xlabel("Year")
plt.ylabel("CO2 Emissions (million metric tons C02)")
plt.savefig("images/Fig8.png")
plt.show()

In [ ]:
em_sectors_combined_df = combined_em_df.drop(columns = ['Total Emissions'])
em_sectors_combined_df.plot(kind="area", title ="Total CO2 Emissions Across Three Sectors")
plt.xlabel("Year")
plt.ylabel("CO2 Emissions (million metric tons C02)")
plt.savefig("images/Fig9.png")
plt.show()

In [ ]:
#Pie chart
em_residential_sum = combined_em_df["Total Residential"].sum()
em_commercial_sum =combined_em_df["Total Commercial"].sum()
em_industrial_sum = combined_em_df["Total Industrial"].sum()

# Labels for the sections of our pie chart
labels = ["Residential", "Commercial", "Industrial"]

# The values of each section of the pie chart
sizes = [em_residential_sum, em_commercial_sum, em_industrial_sum]

# The colors of each section of the pie chart
colors = ["blue", "orange", "green"]

plt.pie(sizes, labels=labels, colors=colors,
        autopct="%1.1f%%", shadow=True, startangle=140)
plt.title("Sectors Contributing to CO2 Emissions in the USA")
plt.axis("equal")
plt.savefig("images/Fig10.png")
plt.show()

# Total Energy Consumption and CO2 emission relationships

In [ ]:
#Create df for scatter plot
total_energy_co2_df = pd.DataFrame({"Total Consumption": total_consumption, "Total Emissions": total_emissions})

In [ ]:
total_energy_co2_df.plot(kind = "scatter", x="Total Consumption", y="Total Emissions", title = "CO2 Emissions by Total Energy Consumption")
plt.show()

In [ ]:
#import lineregress
from scipy.stats import linregress

In [ ]:
x_values = total_energy_co2_df['Total Consumption']
y_values = total_energy_co2_df['Total Emissions']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-") #red solid line
plt.annotate(line_eq,(69729825,1420),fontsize=15,color="red")
plt.xlabel('Total Energy Consumption')
plt.ylabel('Total CO2 emissions')
plt.ylim(1400, 1750)
print(f"The r-squared is: {round(rvalue**2,2)}")
print(f"The actual slope is: {slope}")
plt.savefig("images/Fig11.png")
plt.show()

In [ ]:
x_values = total_energy_co2_df['Total Consumption']/1000000
y_values = total_energy_co2_df['Total Emissions']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-") #red solid line
plt.annotate(line_eq,(69,1420),fontsize=15,color="red")
plt.xlabel('Total Energy Consumption (Billion BTU x 10^6)')
plt.ylabel('Total CO2 emissions')
rsquare = round(rvalue**2,2)
plt.annotate(f"r-squared = {rsquare}", (67.5,1700), fontsize=12)
#plt.ylim(1400, 1750)
print(f"The r-squared is: {round(rvalue**2,2)}")

plt.savefig("images/totenergy_co2_regress.png")
plt.show()

# Solar Energy Consumption

In [ ]:
#Pull in solar energy data files.  Need USA totals, so use the 'uncleaned files'
residential_solar_df = pd.read_csv("data_output/residential_solar_data.csv")
commercial_solar_df = pd.read_csv("data_output/commercial_solar_data.csv")
industrial_solar_df = pd.read_csv("data_output/industrial_solar_data.csv")

In [ ]:
#Set State column as Index
residential_solar_df.set_index("State", inplace = True)
commercial_solar_df.set_index("State", inplace = True)
industrial_solar_df.set_index("State", inplace = True)

In [ ]:
res_solar_US = residential_solar_df.loc[residential_solar_df.index == "USA"]
res_solar_US_T = res_solar_US.T
res_solar_US_T.sort_index(inplace = True,ascending=True)
res_solar_US_T

In [ ]:
#Get commercial energy consumption from USA
com_solar_US = commercial_solar_df.loc[commercial_solar_df.index == "USA"]
com_solar_US_T = com_solar_US.T
com_solar_US_T.sort_index(inplace = True,ascending=True)
com_solar_US_T

In [ ]:
#Get industrial energy from USA
ind_solar_US = industrial_solar_df.loc[industrial_solar_df.index == "USA"]
ind_solar_US_T = ind_solar_US.T
ind_solar_US_T.sort_index(inplace = True,ascending=True)
ind_solar_US_T

In [ ]:
#Combine the two dataframes
combined_solar_df = pd.merge(res_solar_US_T, com_solar_US_T, left_index = True, right_index = True)
combined_solar_df

In [ ]:
#Merge data frames
combined_solar_df = pd.merge(combined_solar_df, ind_solar_US_T, left_index = True, right_index = True)
final_combined_solar_df= combined_solar_df.rename(columns = {'USA_x': 'Total Residential', "USA_y":"Total Commercial", "USA": "Total Industrial"})
final_combined_solar_df.drop(["2016","2017","2018"], inplace = True)
final_combined_solar_df

In [ ]:
final_combined_solar_df.plot(kind="area", title ="Total Solar Energy Consumption Across Three Sectors",)
plt.xlabel("Year")
plt.ylabel("Solar Energy Consumption (Billion BTU)")
plt.savefig("images/solar_area.png", bbox_inches = "tight")

plt.show()

In [ ]:
final_combined_solar_df.plot(kind="line", title ="Total Solar Energy Consumption Across Three Sectors")
plt.xlabel("Year")
plt.ylabel("Solar Energy Consumption (Billion BTU)")
plt.savefig("images/Fig13.png")
plt.show()

In [ ]:
#Sum up the individual sectors to get total consumption
total_combined_solar_df = final_combined_solar_df.copy()
total_combined_solar_df["Total Consumption"] = total_combined_solar_df["Total Residential"] + total_combined_solar_df["Total Commercial"] + total_combined_solar_df["Total Industrial"]
total_combined_solar_df

In [ ]:
total_combined_solar_df.plot(kind="line", title ="Total Solar Energy Consumption Across Three Sectors")
plt.xlabel("Year")
plt.ylabel("Solar Energy Consumption (Billion BTU)")
plt.savefig("images/Fig14.png")
plt.show()

In [ ]:
total_solar_consumption = total_combined_solar_df["Total Consumption"]

total_consumption.plot(kind="line", title ="Total Solar Energy Consumption Across Three Sectors", color = 'r')
plt.xlabel("Year")
plt.ylabel("Consumption (Billion BTUs)")
plt.savefig("images/Fig15.png")
plt.show()

In [ ]:
#Pie chart
residential_solar_sum = final_combined_solar_df["Total Residential"].sum()
commercial_solar_sum =final_combined_solar_df["Total Commercial"].sum()
industrial_solar_sum = final_combined_solar_df["Total Industrial"].sum()

# Labels for the sections of our pie chart
labels = ["Residential", "Commercial", "Industrial"]

# The values of each section of the pie chart
sizes = [residential_solar_sum, commercial_solar_sum, industrial_solar_sum]

# The colors of each section of the pie chart
colors = ["blue", "orange", "green"]

plt.pie(sizes, labels=labels, colors=colors,
        autopct="%1.1f%%", shadow=True, startangle=140)
plt.title("Sectors Contributing to Solar Energy Consumption in the USA")
plt.axis("equal")
plt.savefig("images/Fig16.png")
plt.show()

# Solar Energy Consumption Across the 50 States

In [ ]:
st_residential_solar_df = pd.read_csv("data_output/cleaned_solar_residential.csv")
st_commercial_solar_df = pd.read_csv("data_output/cleaned_solar_commercial.csv")
st_industrial_solar_df = pd.read_csv("data_output/cleaned_solar_industrial.csv")

In [ ]:
#st_residential_solar_df.set_index("State", inplace = True)

st_residential_solar_df
list(st_residential_solar_df.columns)
cleaned_st_residential_solar_df = st_residential_solar_df.drop(columns = ["Unnamed: 0"])
cleaned_st_residential_solar_df

In [ ]:
#Focus on residential since it makes up ~80% of the consumption
df=cleaned_st_residential_solar_df[["2000","2001", "2002","2003","2004","2005", "2006","2007","2008", "2009","2010",
                                 "2011","2012", "2013","2014","2015"]]
boxplot = df.boxplot(figsize=[15.0, 6.0])

plt.title("Distribution of Solar Energy Consumpton per State form 2000-2015")
plt.ylabel("Solar Energy Consumption (Billion BTU)")
plt.savefig("images/Fig17.png")
plt.show()

In [ ]:
df["2015"].sort_values(ascending = False)
#three highest states = CA, FL, AZ
#three lowest states = AK, SD, ND

In [ ]:
df["2015"].median()
#three states around median are: 43 (UT),9(GA),45(VT)

# Analysis of the 3 States with the Highest and Lowest Solar Energy Consumption

In [ ]:
#Pull in total energy data files
residential_df = pd.read_csv("data_output/cleaned_Residential_Energy.csv")
commercial_df = pd.read_csv("data_output/cleaned_Commercial_Energy.csv")
industrial_df = pd.read_csv("data_output/cleaned_Industrial_Energy.csv")

In [ ]:
residential_df.drop(columns=["Unnamed: 0"], inplace = True)
commercial_df.drop(columns=["Unnamed: 0"], inplace = True)
industrial_df.drop(columns=["Unnamed: 0"], inplace = True)

In [ ]:
#Set State column as Index
residential_df.set_index("State", inplace = True)
commercial_df.set_index("State", inplace = True)
industrial_df.set_index("State", inplace = True)

In [ ]:
#Transpose Data
t_residential_df = residential_df.T
t_commercial_df = commercial_df.T
t_industrial_df = industrial_df.T

In [ ]:
t_residential_df.sort_index(inplace = True,ascending=True)
t_commercial_df.sort_index(inplace = True,ascending=True)
t_industrial_df.sort_index(inplace = True,ascending=True)

In [ ]:
t_industrial_df

#3 highest states = CA, FL, AZ
#3 lowest states = AK, SD, ND

Analysis of the 3 States with Highest Solar Energy Consumption (CA, FL, AZ)

In [ ]:
CA_res = t_residential_df["USA-CA"]
CA_com = t_commercial_df["USA-CA"]
CA_ind = t_industrial_df["USA-CA"]

In [ ]:
CA_combined = pd.merge(CA_res, CA_com, left_index = True, right_index = True)
CA_final = pd.merge(CA_combined, CA_ind,left_index = True, right_index = True)
CA_final.columns = ["Residential", "Commercial", "Industrial"]
CA_final["Total Consumption"] = CA_final.sum(axis = 1)
CA_final

In [ ]:
co2_residential_df = pd.read_csv("Resources/residential_c02.csv")
co2_commercial_df = pd.read_csv("Resources/commercial_c02.csv")
co2_industrial_df = pd.read_csv("Resources/industrial_c02.csv")

In [ ]:
#Set 'States' as index
co2_residential_df.set_index("States", inplace = True)
co2_commercial_df.set_index("States", inplace = True)
co2_industrial_df.set_index("States", inplace = True)

In [ ]:
#Transpose dfs
t_co2_residential_df = co2_residential_df.T
t_co2_commercial_df =co2_commercial_df.T
t_co2_industrial_df = co2_industrial_df.T

In [ ]:
CA_co2_res = t_co2_residential_df["CA"]
CA_co2_com = t_co2_commercial_df["CA"]
CA_co2_ind = t_co2_industrial_df["CA"]

In [ ]:
CA_co2_combined = pd.merge(CA_co2_res, CA_co2_com, left_index = True, right_index = True)
CA_co2_final = pd.merge(CA_co2_combined, CA_co2_ind,left_index = True, right_index = True)
CA_co2_final.columns = ["Residential", "Commercial", "Industrial"]
CA_co2_final["Total Emissions"] = CA_co2_final.sum(axis = 1)
CA_co2_final

In [ ]:
CA_consumption = CA_final["Total Consumption"]
CA_emissions = CA_co2_final["Total Emissions"]

In [ ]:
def regression(x_values, y_values):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    print(f"The r-squared is: {round(rvalue**2,3)}")
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    return line_eq, rvalue

In [ ]:
CA_x = CA_final["Total Consumption"]/100000
CA=regression(CA_x, CA_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in CA")
plt.xlabel('Total Energy Consumption (Billon BTU x 10^6)')
plt.ylabel('Total CO2 emissions (million tons CO2)')
plt.annotate(CA[0],(48,105.5),fontsize=15,color="red")
rsquare = round(CA[1]**2,3)
plt.annotate(f"r-squared = {rsquare}", (46.6,114), fontsize=12)
# plt.savefig("images/CA_regress.png")
plt.show()

In [ ]:
CA_x = CA_final["Total Consumption"]/100000
CA=regression(CA_x, CA_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in CA")
plt.xlabel('Total Energy Consumption')
plt.ylabel('Total CO2 emissions')
plt.annotate(CA,(48,16),fontsize=15,color="red")
rsquare = round(rvalue**2,2)
plt.annotate(f"r-squared = {rsquare}", (47,114), fontsize=12)
#plt.savefig("images/Fig19.png")
plt.show()

In [ ]:
x_values = CA_final["Total Consumption"]
y_values = CA_co2_final["Total Emissions"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-") #red solid line
plt.annotate(line_eq,(4800000,105),fontsize=15,color="red")
plt.title("CO2 emission by Total Energy Consumption in CA")
plt.xlabel('Total Energy Consumption')
plt.ylabel('Total CO2 emissions')

print(f"The r-squared is: {round(rvalue**2,4)}")
print(f"The actual slope is: {slope}")
# plt.savefig("images/Fig18.png")
plt.show()

In [ ]:
FL_res = t_residential_df["USA-FL"]
FL_com = t_commercial_df["USA-FL"]
FL_ind = t_industrial_df["USA-FL"]
FL_combined = pd.merge(FL_res, FL_com, left_index = True, right_index = True)
FL_final = pd.merge(FL_combined, FL_ind,left_index = True, right_index = True)
FL_final.columns = ["Residential", "Commercial", "Industrial"]
FL_final["Total Consumption"] = FL_final.sum(axis = 1)
FL_final

In [ ]:
FL_co2_res = t_co2_residential_df["FL"]
FL_co2_com = t_co2_commercial_df["FL"]
FL_co2_ind = t_co2_industrial_df["FL"]

FL_co2_combined = pd.merge(FL_co2_res, FL_co2_com, left_index = True, right_index = True)
FL_co2_final = pd.merge(FL_co2_combined, FL_co2_ind,left_index = True, right_index = True)
FL_co2_final.columns = ["Residential", "Commercial", "Industrial"]
FL_co2_final["Total Emissions"] = FL_co2_final.sum(axis = 1)
FL_co2_final

In [ ]:
FL_x = FL_final["Total Consumption"]/1000000
FL=regression(FL_x, FL_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in FL")
plt.xlabel('Total Energy Consumption (billion BTU x 10^6)')
plt.ylabel('Total CO2 Emissions (million tons CO2)')
plt.annotate(FL[0],(2.72,15.5),fontsize=15,color="red")
rsquare = round(FL[1]**2,3)
plt.annotate(f"r-squared = {rsquare}", (2.55,19.5), fontsize=12)
# plt.savefig("images/FL_reg.png")
plt.show()

In [ ]:
AZ_res = t_residential_df["USA-AZ"]
AZ_com = t_commercial_df["USA-AZ"]
AZ_ind = t_industrial_df["USA-AZ"]
AZ_combined = pd.merge(AZ_res, AZ_com, left_index = True, right_index = True)
AZ_final = pd.merge(AZ_combined, AZ_ind,left_index = True, right_index = True)
AZ_final.columns = ["Residential", "Commercial", "Industrial"]
AZ_final["Total Consumption"] = AZ_final.sum(axis = 1)
AZ_final

In [ ]:
AZ_co2_res = t_co2_residential_df["AZ"]
AZ_co2_com = t_co2_commercial_df["AZ"]
AZ_co2_ind = t_co2_industrial_df["AZ"]

AZ_co2_combined = pd.merge(AZ_co2_res, AZ_co2_com, left_index = True, right_index = True)
AZ_co2_final = pd.merge(AZ_co2_combined, AZ_co2_ind,left_index = True, right_index = True)
AZ_co2_final.columns = ["Residential", "Commercial", "Industrial"]
AZ_co2_final["Total Emissions"] = AZ_co2_final.sum(axis = 1)
AZ_co2_final

In [ ]:
AZ_x = AZ_final["Total Consumption"]/1000000
AZ=regression(AZ_x, AZ_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in AZ")
plt.xlabel('Total Energy Consumption (billion BTU x10^6)')
plt.ylabel('Total CO2 emissions (million tons CO2)')
plt.annotate(AZ[0],(0.9,7.8),fontsize=15,color="red")
rsquare = round(AZ[1]**2,3)
plt.annotate(f"r-squared = {rsquare}", (0.86,9.5), fontsize=12)
# plt.savefig("images/AZ_reg.png")
plt.show()

r-squared USA total = 0.52 CA r-squared = 0.4952 FL r-squared = 0.205 AZ r-squared = 0.46 For the 3 states with the highest solar energy consumption, their r2 is < that for all of USA (although CA and AZ probably aren't significantly different).

Analysis of the 3 States with Lowest Solar Energy Consumption (AK, SD, ND)

In [ ]:
AK_res = t_residential_df["USA-AK"]
AK_com = t_commercial_df["USA-AK"]
AK_ind = t_industrial_df["USA-AK"]
AK_combined = pd.merge(AK_res, AK_com, left_index = True, right_index = True)
AK_final = pd.merge(AK_combined, AK_ind,left_index = True, right_index = True)
AK_final.columns = ["Residential", "Commercial", "Industrial"]
AK_final["Total Consumption"] = AK_final.sum(axis = 1)
AK_final

In [ ]:
AK_co2_res = t_co2_residential_df["AK"]
AK_co2_com = t_co2_commercial_df["AK"]
AK_co2_ind = t_co2_industrial_df["AK"]

AK_co2_combined = pd.merge(AK_co2_res, AK_co2_com, left_index = True, right_index = True)
AK_co2_final = pd.merge(AK_co2_combined, AK_co2_ind,left_index = True, right_index = True)
AK_co2_final.columns = ["Residential", "Commercial", "Industrial"]
AK_co2_final["Total Emissions"] = AK_co2_final.sum(axis = 1)
AK_co2_final

In [ ]:
AK_x = AK_final["Total Consumption"]/1000000
AK=regression(AK_x, AK_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in AK")
plt.xlabel('Total Energy Consumption (billion BTU x10^6)')
plt.ylabel('Total CO2 emissions (million tons CO2)')
plt.annotate(AK[0],(0.48,21),fontsize=15,color="red")
rsquare = round(AK[1]**2,3)
plt.annotate(f"r-squared = {rsquare}", (0.44,25), fontsize=12)
plt.savefig("images/AK_reg.png")
plt.show()

In [ ]:
SD_res = t_residential_df["USA-SD"]
SD_com = t_commercial_df["USA-SD"]
SD_ind = t_industrial_df["USA-SD"]
SD_combined = pd.merge(SD_res, SD_com, left_index = True, right_index = True)
SD_final = pd.merge(SD_combined, SD_ind,left_index = True, right_index = True)
SD_final.columns = ["Residential", "Commercial", "Industrial"]
SD_final["Total Consumption"] = SD_final.sum(axis = 1)
SD_final

In [ ]:
SD_co2_res = t_co2_residential_df["SD"]
SD_co2_com = t_co2_commercial_df["SD"]
SD_co2_ind = t_co2_industrial_df["SD"]

SD_co2_combined = pd.merge(SD_co2_res, SD_co2_com, left_index = True, right_index = True)
SD_co2_final = pd.merge(SD_co2_combined, SD_co2_ind,left_index = True, right_index = True)
SD_co2_final.columns = ["Residential", "Commercial", "Industrial"]
SD_co2_final["Total Emissions"] = SD_co2_final.sum(axis = 1)
SD_co2_final

In [ ]:
SD_x=SD_final["Total Consumption"]/1000000
SD=regression(SD_x, SD_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in SD")
plt.xlabel('Total Energy Consumption (billion BTU x10^6)')
plt.ylabel('Total CO2 emissions (million tons CO2)')
plt.annotate(SD[0],(0.24,3.8),fontsize=15,color="red")
rsquare = round(SD[1]**2,3)
plt.annotate(f"r-squared = {rsquare}", (0.18,5.5), fontsize=12)
# plt.savefig("images/SD_reg.png")
plt.show()

In [ ]:
ND_res = t_residential_df["USA-ND"]
ND_com = t_commercial_df["USA-ND"]
ND_ind = t_industrial_df["USA-ND"]
ND_combined = pd.merge(ND_res, ND_com, left_index = True, right_index = True)
ND_final = pd.merge(ND_combined, ND_ind,left_index = True, right_index = True)
ND_final.columns = ["Residential", "Commercial", "Industrial"]
ND_final["Total Consumption"] = ND_final.sum(axis = 1)
ND_final

In [ ]:
ND_co2_res = t_co2_residential_df["ND"]
ND_co2_com = t_co2_commercial_df["ND"]
ND_co2_ind = t_co2_industrial_df["ND"]

ND_co2_combined = pd.merge(ND_co2_res, ND_co2_com, left_index = True, right_index = True)
ND_co2_final = pd.merge(ND_co2_combined, ND_co2_ind,left_index = True, right_index = True)
ND_co2_final.columns = ["Residential", "Commercial", "Industrial"]
ND_co2_final["Total Emissions"] = ND_co2_final.sum(axis = 1)
ND_co2_final

In [ ]:
ND_x = ND_final["Total Consumption"]/1000000
ND=regression(ND_x, ND_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in ND")
plt.xlabel('Total Energy Consumption (billion BTU x10^6)')
plt.ylabel('Total CO2 emissions (million tons CO2)')
plt.annotate(ND[0],(0.40,14),fontsize=15,color="red")
rsquare = round(ND[1]**2,3)
plt.annotate(f"r-squared = {rsquare}", (0.31,19), fontsize=12)                  
plt.savefig("images/ND_reg.png")
plt.show()

r-squared for US = 0.52 r-squared for AK = 0.98 r-squared for SD = 0.789 r-squared for ND = 0.96 For the three states with the lowest solar energy consumption, the r-squared for CO2 emissions by Total Energy Consumption are very high (0.79 -0.98), suggesting that the regression models for these states account for the majority of the variance in the data observed.

# Analysis of the 3 States with Median Solar Energy Consumption (GA, UT, VT)

In [ ]:
GA_res = t_residential_df["USA-GA"]
GA_com = t_commercial_df["USA-GA"]
GA_ind = t_industrial_df["USA-GA"]
GA_combined = pd.merge(GA_res, GA_com, left_index = True, right_index = True)
GA_final = pd.merge(GA_combined, GA_ind,left_index = True, right_index = True)
GA_final.columns = ["Residential", "Commercial", "Industrial"]
GA_final["Total Consumption"] = GA_final.sum(axis = 1)
GA_final

In [ ]:
GA_co2_res = t_co2_residential_df["GA"]
GA_co2_com = t_co2_commercial_df["GA"]
GA_co2_ind = t_co2_industrial_df["GA"]

GA_co2_combined = pd.merge(GA_co2_res, GA_co2_com, left_index = True, right_index = True)
GA_co2_final = pd.merge(GA_co2_combined, GA_co2_ind,left_index = True, right_index = True)
GA_co2_final.columns = ["Residential", "Commercial", "Industrial"]
GA_co2_final["Total Emissions"] = GA_co2_final.sum(axis = 1)
GA_co2_final

In [ ]:
GA=regression(GA_final["Total Consumption"], GA_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in GA")
plt.xlabel('Total Energy Consumption')
plt.ylabel('Total CO2 emissions')
plt.annotate(GA,(2020000,22),fontsize=15,color="red")
plt.savefig("images/Fig24.png")
plt.show()

In [ ]:
UT_res = t_residential_df["USA-UT"]
UT_com = t_commercial_df["USA-UT"]
UT_ind = t_industrial_df["USA-UT"]
UT_combined = pd.merge(UT_res, UT_com, left_index = True, right_index = True)
UT_final = pd.merge(UT_combined, UT_ind,left_index = True, right_index = True)
UT_final.columns = ["Residential", "Commercial", "Industrial"]
UT_final["Total Consumption"] = UT_final.sum(axis = 1)
UT_final

In [ ]:
UT_co2_res = t_co2_residential_df["UT"]
UT_co2_com = t_co2_commercial_df["UT"]
UT_co2_ind = t_co2_industrial_df["UT"]

UT_co2_combined = pd.merge(UT_co2_res, UT_co2_com, left_index = True, right_index = True)
UT_co2_final = pd.merge(UT_co2_combined, UT_co2_ind,left_index = True, right_index = True)
UT_co2_final.columns = ["Residential", "Commercial", "Industrial"]
UT_co2_final["Total Emissions"] = UT_co2_final.sum(axis = 1)
UT_co2_final

In [ ]:
UT=regression(UT_final["Total Consumption"], UT_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in UT")
plt.xlabel('Total Energy Consumption')
plt.ylabel('Total CO2 emissions')
plt.annotate(UT,(530000,11.5),fontsize=15,color="red")
plt.savefig("images/Fig25.png")
plt.show()

In [ ]:
VT_res = t_residential_df["USA-VT"]
VT_com = t_commercial_df["USA-VT"]
VT_ind = t_industrial_df["USA-VT"]
VT_combined = pd.merge(VT_res, VT_com, left_index = True, right_index = True)
VT_final = pd.merge(VT_combined, VT_ind,left_index = True, right_index = True)
VT_final.columns = ["Residential", "Commercial", "Industrial"]
VT_final["Total Consumption"] = VT_final.sum(axis = 1)
VT_final

In [ ]:
VT_co2_res = t_co2_residential_df["VT"]
VT_co2_com = t_co2_commercial_df["VT"]
VT_co2_ind = t_co2_industrial_df["VT"]

VT_co2_combined = pd.merge(VT_co2_res, VT_co2_com, left_index = True, right_index = True)
VT_co2_final = pd.merge(VT_co2_combined, VT_co2_ind,left_index = True, right_index = True)
VT_co2_final.columns = ["Residential", "Commercial", "Industrial"]
VT_co2_final["Total Emissions"] = VT_co2_final.sum(axis = 1)
VT_co2_final

In [ ]:
VT=regression(VT_final["Total Consumption"], VT_co2_final["Total Emissions"])
plt.title("CO2 emission by Total Energy Consumption in VT")
plt.xlabel('Total Energy Consumption')
plt.ylabel('Total CO2 emissions')
plt.annotate(VT,(100000,2.2),fontsize=15,color="red")
plt.savefig("images/Fig26.png")
plt.show()

Taken together, the regression models suggest that in states with the lowest solar energy consumption, the variability in CO2 emissions is mostly accounted for by the total energy consumption in that state. In states with the hightest solar energy consumption, the variablity in CO2 emissions is inadequately accounted for by total energy consumption.

In [ ]:
VT_final

In [ ]:
VT_final["Res %"] = VT_final["Total Consumption"]/VT_final["Residential"]
VT_final

# Further CO2 Analysis

In [ ]:
#Total Carbon Dioxide Emissions, all fuels, in millions metric tons
commercial = pd.read_csv('c02_df_data/commercial_c02.csv')
industrial = pd.read_csv('c02_df_data/industrial_c02.csv')
residential = pd.read_csv('c02_df_data/residential_c02.csv')

In [ ]:
years = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
compare_dict = {'Commercial': [commercial['2000'].sum(),commercial['2001'].sum(),commercial['2002'].sum(),
                       commercial['2003'].sum(),commercial['2004'].sum(),commercial['2005'].sum(),
                       commercial['2004'].sum(),commercial['2007'].sum(),commercial['2008'].sum(),
                       commercial['2009'].sum(),commercial['2010'].sum(),commercial['2011'].sum(),
                       commercial['2012'].sum(),commercial['2013'].sum(),commercial['2014'].sum(),
                        commercial['2015'].sum()],
       'Industrial' : [industrial['2000'].sum(),industrial['2001'].sum(),industrial['2002'].sum(),
                       industrial['2003'].sum(),industrial['2004'].sum(),industrial['2005'].sum(),
                       industrial['2004'].sum(),industrial['2007'].sum(),industrial['2008'].sum(),
                       industrial['2009'].sum(),industrial['2010'].sum(),industrial['2011'].sum(),
                       industrial['2012'].sum(),industrial['2013'].sum(),industrial['2014'].sum(),
                          industrial['2015'].sum()],
       'Residential' :[residential['2000'].sum(),residential['2001'].sum(),residential['2002'].sum(),
                       residential['2003'].sum(),residential['2004'].sum(),residential['2005'].sum(),
                       residential['2004'].sum(),residential['2007'].sum(),residential['2008'].sum(),
                       residential['2009'].sum(),residential['2010'].sum(),residential['2011'].sum(),
                       residential['2012'].sum(),residential['2013'].sum(),residential['2014'].sum(),
                          residential['2015'].sum()],
               'Year': years}

all_df = pd.DataFrame(data = compare_dict).reset_index()
all_df['Total'] = all_df['Commercial'] + all_df['Industrial'] + all_df['Residential']
all_df

In [ ]:
#plt.plot(all_df['Year'],all_df['Commercial'])
#plt.plot(all_df['Year'],all_df['Industrial'], color = 'r')
plt.plot(all_df['Year'],all_df['Total'], color = 'b')
plt.ylim(1000,2000)
plt.show()

In [ ]:
commercial

In [ ]:
residential